# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 2:11PM,256969,19,SNAP28231,"Snap Medical Industries, LLC",Released
1,Feb 8 2023 2:10PM,256968,19,9132997,VITABIOTICS USA INC.,Released
2,Feb 8 2023 2:04PM,256967,19,9132983,VITABIOTICS USA INC.,Released
3,Feb 8 2023 2:00PM,256966,10,9132986,"Snap Medical Industries, LLC",Released
4,Feb 8 2023 1:58PM,256962,10,0086364960,ISDIN Corporation,Released
5,Feb 8 2023 1:58PM,256962,10,0086364949,ISDIN Corporation,Released
6,Feb 8 2023 1:58PM,256962,10,0086364950,ISDIN Corporation,Released
7,Feb 8 2023 1:58PM,256962,10,0086364951,ISDIN Corporation,Released
8,Feb 8 2023 1:58PM,256962,10,0086364955,ISDIN Corporation,Released
9,Feb 8 2023 1:58PM,256962,10,0086364956,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,256965,Released,2
55,256966,Released,1
56,256967,Released,1
57,256968,Released,1
58,256969,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256965,NaN,NaN,2.0
256966,NaN,NaN,1.0
256967,NaN,NaN,1.0
256968,NaN,NaN,1.0
256969,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,45.0,2.0,5.0
256826,0.0,5.0,108.0
256827,13.0,1.0,2.0
256857,0.0,1.0,10.0
256868,5.0,21.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,45,2,5
256826,0,5,108
256827,13,1,2
256857,0,1,10
256868,5,21,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,45,2,5
1,256826,0,5,108
2,256827,13,1,2
3,256857,0,1,10
4,256868,5,21,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,45,2,5
1,256826,,5,108
2,256827,13,1,2
3,256857,,1,10
4,256868,5,21,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 2:11PM,256969,19,"Snap Medical Industries, LLC"
1,Feb 8 2023 2:10PM,256968,19,VITABIOTICS USA INC.
2,Feb 8 2023 2:04PM,256967,19,VITABIOTICS USA INC.
3,Feb 8 2023 2:00PM,256966,10,"Snap Medical Industries, LLC"
4,Feb 8 2023 1:58PM,256962,10,ISDIN Corporation
22,Feb 8 2023 1:57PM,256960,19,Eywa Pharma Inc.
25,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc."
27,Feb 8 2023 1:50PM,256964,10,Emerginnova
28,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation
46,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 2:11PM,256969,19,"Snap Medical Industries, LLC",,,1
1,Feb 8 2023 2:10PM,256968,19,VITABIOTICS USA INC.,,,1
2,Feb 8 2023 2:04PM,256967,19,VITABIOTICS USA INC.,,,1
3,Feb 8 2023 2:00PM,256966,10,"Snap Medical Industries, LLC",,,1
4,Feb 8 2023 1:58PM,256962,10,ISDIN Corporation,,,18
5,Feb 8 2023 1:57PM,256960,19,Eywa Pharma Inc.,,,3
6,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",,,2
7,Feb 8 2023 1:50PM,256964,10,Emerginnova,,,1
8,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,,,18
9,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 2:11PM,256969,19,"Snap Medical Industries, LLC",1,,
1,Feb 8 2023 2:10PM,256968,19,VITABIOTICS USA INC.,1,,
2,Feb 8 2023 2:04PM,256967,19,VITABIOTICS USA INC.,1,,
3,Feb 8 2023 2:00PM,256966,10,"Snap Medical Industries, LLC",1,,
4,Feb 8 2023 1:58PM,256962,10,ISDIN Corporation,18,,
5,Feb 8 2023 1:57PM,256960,19,Eywa Pharma Inc.,3,,
6,Feb 8 2023 1:54PM,256965,10,"Methapharm, Inc.",2,,
7,Feb 8 2023 1:50PM,256964,10,Emerginnova,1,,
8,Feb 8 2023 1:49PM,256963,10,ISDIN Corporation,18,,
9,Feb 8 2023 1:45PM,256961,19,"AdvaGen Pharma, Ltd",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 2:11PM,256969,19,"Snap Medical Industries, LLC",1,,
1,Feb 8 2023 2:10PM,256968,19,VITABIOTICS USA INC.,1,,
2,Feb 8 2023 2:04PM,256967,19,VITABIOTICS USA INC.,1,,
3,Feb 8 2023 2:00PM,256966,10,"Snap Medical Industries, LLC",1,,
4,Feb 8 2023 1:58PM,256962,10,ISDIN Corporation,18,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 2:11PM,256969,19,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Feb 8 2023 2:10PM,256968,19,VITABIOTICS USA INC.,1.0,NaN,NaN
2,Feb 8 2023 2:04PM,256967,19,VITABIOTICS USA INC.,1.0,NaN,NaN
3,Feb 8 2023 2:00PM,256966,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Feb 8 2023 1:58PM,256962,10,ISDIN Corporation,18.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 2:11PM,256969,19,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Feb 8 2023 2:10PM,256968,19,VITABIOTICS USA INC.,1.0,0.0,0.0
2,Feb 8 2023 2:04PM,256967,19,VITABIOTICS USA INC.,1.0,0.0,0.0
3,Feb 8 2023 2:00PM,256966,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Feb 8 2023 1:58PM,256962,10,ISDIN Corporation,18.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6166418,136.0,5.0,0.0
12,256956,4.0,0.0,0.0
15,1284271,119.0,23.0,74.0
16,256873,0.0,1.0,0.0
19,3597048,26.0,23.0,5.0
20,256930,1.0,0.0,0.0
21,256870,1.0,0.0,0.0
22,256918,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6166418,136.0,5.0,0.0
1,12,256956,4.0,0.0,0.0
2,15,1284271,119.0,23.0,74.0
3,16,256873,0.0,1.0,0.0
4,19,3597048,26.0,23.0,5.0
5,20,256930,1.0,0.0,0.0
6,21,256870,1.0,0.0,0.0
7,22,256918,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,136.0,5.0,0.0
1,12,4.0,0.0,0.0
2,15,119.0,23.0,74.0
3,16,0.0,1.0,0.0
4,19,26.0,23.0,5.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,136.0
1,12,Released,4.0
2,15,Released,119.0
3,16,Released,0.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,74.0,0.0,5.0,0.0,0.0,0.0
Executing,5.0,0.0,23.0,1.0,23.0,0.0,0.0,0.0
Released,136.0,4.0,119.0,0.0,26.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,74.0,0.0,5.0,0.0,0.0,0.0
1,Executing,5.0,0.0,23.0,1.0,23.0,0.0,0.0,0.0
2,Released,136.0,4.0,119.0,0.0,26.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,74.0,0.0,5.0,0.0,0.0,0.0
1,Executing,5.0,0.0,23.0,1.0,23.0,0.0,0.0,0.0
2,Released,136.0,4.0,119.0,0.0,26.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()